**Task-1**

In [1]:
def min_edit_distance(s1, s2):

    m = len(s1)
    n = len(s2)


    dp = [[0] * (n + 1) for _ in range(m + 1)]


    for i in range(m + 1):
        dp[i][0] = i

    for j in range(n + 1):
        dp[0][j] = j


    for i in range(1, m + 1):
        for j in range(1, n + 1):


            cost = 0 if s1[i - 1] == s2[j - 1] else 1


            dp[i][j] = min(dp[i - 1][j] + 1,
                           dp[i][j - 1] + 1,
                           dp[i - 1][j - 1] + cost)


    return dp[m][n]

In [ ]:
def similarity_score(str1, str2):
    distance = min_edit_distance(str1, str2)
    max_len = max(len(str1), len(str2))
    if max_len == 0:
        return 1.0
    return 1 - (distance / max_len)

def main():
    print("String Similarity Calculator")
    str1 = input("Enter first string: ")
    str2 = input("Enter second string: ")

    score = similarity_score(str1, str2)
    print(f"\nSimilarity Score: {score:.2f}")

if __name__ == "__main__":
    main()

String Similarity Calculator (with DP Table)


**Task-2**

Setup and Load Corpus

In [ ]:
import nltk
from nltk.corpus import brown
import os


nltk.download('brown')
nltk.download('punkt')
nltk.download('punkt_tab')



sentences = brown.sents()
sentences = [[w.lower() for w in sent] for sent in sentences]

print("Total sentences in Brown Corpus:", len(sentences))
print("Example sentence:", sentences[0])

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\88017\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\brown.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\88017\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\88017\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


Total sentences in Brown Corpus: 57340
Example sentence: ['the', 'fulton', 'county', 'grand', 'jury', 'said', 'friday', 'an', 'investigation', 'of', "atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.']


Build unigram, bigram, trigram counts

In [8]:
from collections import Counter
#Counter is a special dictionary in Python that counts how many times each item appears

unigram_counts = Counter()
bigram_counts = Counter()
trigram_counts = Counter()

for sent in sentences:
    sent = ["<s>"] + sent + ["</s>"]
    for i in range(len(sent)):
        unigram_counts[sent[i]] += 1
    for i in range(len(sent)-1):
        bigram_counts[(sent[i], sent[i+1])] += 1
    for i in range(len(sent)-2):
        trigram_counts[(sent[i], sent[i+1], sent[i+2])] += 1

vocab = len(unigram_counts)

most_freq_word = unigram_counts.most_common(1)[0][0]

print("Vocabulary size:", vocab)
print("Most frequent word:", most_freq_word)

Vocabulary size: 49817
Most frequent word: the


Predict next word safely

In [9]:
def predict_next_word_bigram(word):
    candidates = {w2: count for (w1, w2), count in bigram_counts.items() if w1 == word}
    if candidates:
        return max(candidates, key=candidates.get)
    else:
        return most_freq_word

def predict_next_word_trigram(word1, word2):
    candidates = {w3: count for (w1, w2_, w3), count in trigram_counts.items() if w1 == word1 and w2_ == word2}
    if candidates:
        return max(candidates, key=candidates.get)
    else:
        return predict_next_word_bigram(word2)

Show n-grams

In [10]:


def show_ngrams(sentence):

    nltk.data.path.append(os.path.join(os.path.expanduser("~"), "nltk_data"))
    tokens = nltk.word_tokenize(sentence.lower())
    bigrams = list(nltk.bigrams(tokens))
    trigrams = list(nltk.trigrams(tokens))
    return bigrams, trigrams


sentence = "i am eating rice"
bigrams, trigrams = show_ngrams(sentence)
print("Bigrams:", bigrams)
print("Trigrams:", trigrams)

Bigrams: [('i', 'am'), ('am', 'eating'), ('eating', 'rice')]
Trigrams: [('i', 'am', 'eating'), ('am', 'eating', 'rice')]


Sentence probability using plain Laplace smoothing

In [11]:
def bigram_probability_laplace(sentence):
    tokens = ["<s>"] + nltk.word_tokenize(sentence.lower()) + ["</s>"]
    prob = 1.0
    for i in range(len(tokens)-1):
        w1, w2 = tokens[i], tokens[i+1]
        count_bigram = bigram_counts.get((w1, w2), 0)
        count_unigram = unigram_counts.get(w1, 0)
        prob *= (count_bigram + 1) / (count_unigram + vocab)
    return prob

def trigram_probability_laplace(sentence):
    tokens = ["<s>"] + nltk.word_tokenize(sentence.lower()) + ["</s>"]
    prob = 1.0
    for i in range(len(tokens)-2):
        w1, w2, w3 = tokens[i], tokens[i+1], tokens[i+2]
        count_trigram = trigram_counts.get((w1, w2, w3), 0)
        count_bigram = bigram_counts.get((w1, w2), 0)
        prob *= (count_trigram + 1) / (count_bigram + vocab)
    return prob

User Input Loop and probability using laplace smothing

In [12]:
while True:
    print("\nEnter a sentence (or type 'exit' to quit):")
    user_input = input()
    if user_input.lower() == "exit":
        break

    tokens = nltk.word_tokenize(user_input.lower())
    print("Tokens:", tokens)


    if len(tokens) >= 2:
        next_word = predict_next_word_trigram(tokens[-2], tokens[-1])
    else:
        next_word = predict_next_word_bigram(tokens[-1])
    print("Predicted next word:", next_word)


    bigrams, trigrams = show_ngrams(user_input)
    print("Bigrams:", bigrams)
    print("Trigrams:", trigrams)


    bigram_prob = bigram_probability_laplace(user_input)
    trigram_prob = trigram_probability_laplace(user_input)
    print(f"Bigram Probability (Laplace): {bigram_prob:.10e}")
    print(f"Trigram Probability (Laplace): {trigram_prob:.10e}")


Enter a sentence (or type 'exit' to quit):
Tokens: ['rafi']
Predicted next word: the
Bigrams: []
Trigrams: []
Bigram Probability (Laplace): 1.8732764911e-10
Trigram Probability (Laplace): 2.0073468896e-05

Enter a sentence (or type 'exit' to quit):
Tokens: ['tafi']
Predicted next word: the
Bigrams: []
Trigrams: []
Bigram Probability (Laplace): 1.8732764911e-10
Trigram Probability (Laplace): 2.0073468896e-05

Enter a sentence (or type 'exit' to quit):
Tokens: ['boo']
Predicted next word: you
Bigrams: []
Trigrams: []
Bigram Probability (Laplace): 1.8732388887e-10
Trigram Probability (Laplace): 2.0073468896e-05

Enter a sentence (or type 'exit' to quit):
Tokens: ['soo']
Predicted next word: the
Bigrams: []
Trigrams: []
Bigram Probability (Laplace): 1.8732764911e-10
Trigram Probability (Laplace): 2.0073468896e-05

Enter a sentence (or type 'exit' to quit):
Tokens: []


IndexError: list index out of range